## 在运行本笔记本之前，请先下载以下工具：

1. **查找当前的 Google Chrome 版本**  
   - 打开 Google Chrome，并在地址栏输入：  
     ```
     chrome://settings/help
     ```
   - 建议使用 **133** 版本的 Google Chrome（或最新可用的稳定版本）。

2. **下载对应版本的 ChromeDriver**  
   - 前往官方的 Chrome 测试下载页面：  
     [https://googlechromelabs.github.io/chrome-for-testing/#stable](https://googlechromelabs.github.io/chrome-for-testing/#stable)  
   - 下载与您的 Chrome 版本相匹配的 ChromeDriver。  
   - 例如，如果您的 Chrome 版本是 133，请下载 **ChromeDriver 133**。

3. **查找已下载的 ChromeDriver**  
   - 如果您使用 macOS，可在终端 (Terminal) 中运行以下命令来查找 `chromedriver` 的位置：
     ```bash
     mdfind -name chromedriver
     ```
   - 如果使用其他操作系统，请检查默认的 **下载** 目录或您保存该文件的目录。

In [24]:
# 导入必要的包
import os
import re
import time
import json
import random
import pymongo
import pandas as pd
import requests  # 新增：用于下载图片

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

##### 启动 Chrome 浏览器实例：

打开 **terminal**, 下载 Chrome Driver (假的 Google Chrome)

```bash
brew install chromedriver
chmod +x /opt/homebrew/bin/chromedriver
```

输入以下命令（将 `your Chrome.exe path` 替换为您的 Google Chrome 浏览器路径）：
```bash
<your Chrome.exe path> --remote-debugging-port=9222 --user-data-dir="/Users/<your home folder name>/selenium/AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)
- 来做个比方， 我的 *terminal command* 会是:

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir="/Users/princess/selenium/AutomationProfile"

In [25]:
# 配置Chrome浏览器
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')
browser = webdriver.Chrome(options=options)
action = ActionChains(browser)

In [26]:
user_url = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_user_page_load_status(browser):
    """
    检查用户主页是否加载成功。
    这里等待页面中出现至少一个帖子（包含 "note-item" 的 section）。
    """
    print("检查用户主页加载状态...")
    try:
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@class, "note-item")]')))
        print("用户主页加载成功")
    except Exception as e:
        print("用户主页加载超时或出错:", e)

def selenium_test():
    """
    登录状态检查、页面加载检查，并根据用户输入跳转到指定的用户主页进行爬取
    """
    global user_url, num
    # 首先打开探索页以触发登录检查
    browser.get('https://www.xiaohongshu.com/explore')
    check_login_status(browser)
    
    print("请根据提示输入用户主页URL和笔记爬取数量。")
    user_url = input("用户主页URL：").strip()
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    # 跳转到指定用户主页
    browser.get(user_url)
    time.sleep(3)
    check_user_page_load_status(browser)

# 调用 selenium_test() 进行初始化（确保此处 browser 已创建）
selenium_test()

即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
登录成功
检查时间: Tue Feb 11 20:47:00 2025
请根据提示输入用户主页URL和笔记爬取数量。
检查用户主页加载状态...
用户主页加载成功


In [27]:
# 初始化数据存储列表
authorName_list = []
likeNr_list = []
id_list = []
noteURL_list = []
commentNr_list = []
post_title_list = [] 
caption_list = []  # 存储帖子内容
datePublished_list = []
images_list = []
author_avatar_list = []  # 用于存储头像图片
starNr_list = []
authorCollectNr_list = []
authorFansNr_list = []
authorNoteNr_list = []
video_urls = [] 

def parseUserPage(page_source):
    """
    解析当前页面的HTML内容，提取用户笔记的基本信息并更新对应的列表。
    同时，利用页面中提取一次作者的个人数据（收藏、粉丝和笔记数量）
    来填充每个笔记记录对应的作者数据。
    
    Args:
        page_source (str): 当前页面的HTML内容
    """
    response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
    selector = Selector(response)

    print("正在分析页面结构...")

    content_elements = selector.xpath('//section[contains(@class, "note-item")]')
    if content_elements:
        print(f"找到 {len(content_elements)} 个内容元素")
        for element in content_elements:
            try:
                # 提取帖子（笔记）的相对URL及 note_id
                note_url = element.xpath('.//a[contains(@class, "cover")]/@href').get()
                if note_url:
                    note_id = note_url.split('/')[-1].split('?')[0]
                    if note_id in id_list:
                        continue  # 避免重复爬取
                    id_list.append(note_id)
                    noteURL_list.append(note_url)

                    # 提取作者名字（通常与用户主页相同）
                    author = element.xpath('.//div[contains(@class, "author-wrapper")]//span[contains(@class, "name")]/text()').get()
                    authorName_list.append(author.strip() if author else "N/A")

                    # 提取点赞数量
                    likes = element.xpath('.//span[contains(@class, "like-wrapper")]/span[contains(@class, "count")]/text()').get()
                    likeNr_list.append(likes.strip() if likes else "0")

                    # 提取帖子标题
                    post_title = element.xpath('.//a[contains(@class, "title")]//span/text()').getall()
                    post_title_cleaned = ' '.join([c.strip() for c in post_title if c.strip()])
                    post_title_list.append(post_title_cleaned if post_title_cleaned else "N/A")

                    # 提取图片（主图）
                    main_image = element.xpath('.//a[contains(@class, "cover")]/img/@src').get()
                    images_list.append(main_image.strip() if main_image else "N/A")

                    # 提取头像图片
                    avatar_image = element.xpath('.//a[contains(@class, "author")]/img/@src').get()
                    author_avatar_list.append(avatar_image.strip() if avatar_image else "N/A")

                    # 选择并提取网页数据
                    author_collect_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[3]/span[@class="count"]/text()').extract_first()# 作者获赞与收藏数量
                    author_fans_nr = selector.xpath('//*[@class="data-info"]/div[1]/div[2]/span[@class="count"]/text()').extract_first()# 作者粉丝数量
                    author_note_nr = len(selector.xpath('//*[@id="userPostedFeeds"]//section'))# 作者笔记数量

                    authorCollectNr_list.append(author_collect_nr)
                    authorFansNr_list.append(author_fans_nr)
                    authorNoteNr_list.append(author_note_nr)

                    # 初始化附加字段：
                    # note‑specific fields (后面会在 parseNotePage 中更新)
                    commentNr_list.append("0")
                    datePublished_list.append("N/A")
                    starNr_list.append("0")
                    video_urls.append("N/A")
                    caption_list.append("N/A")
                    
                    qbar.update(1)
            except Exception as e:
                print(f"处理元素时出错: {str(e)}")
                continue

    print(f"当前已爬取总数: {len(id_list)}")

def parseNotePage(note_url, note_id):
    """
    访问每个笔记的页面，提取附加字段，包括评论数量、发布时间、收藏数量（针对笔记）、
    视频 URL 以及帖子内容（Caption）。
    注意：原来提取作者个人数据的代码已移入 parseUserPage。
    
    Args:
        note_url (str): 笔记的相对URL
        note_id (str): 笔记的唯一ID
    """
    try:
        full_note_url = f'https://www.xiaohongshu.com{note_url}'
        browser.get(full_note_url)
        
        # 等待页面加载中关键的描述 meta 标签
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))
        
        page_source = browser.page_source
        response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
        selector = Selector(response)

        # 提取评论数量
        comments = selector.xpath('//*[@class="total"]/text()').get()
        comments = comments.strip() if comments else "0"

        # 提取发布时间（采用多种 XPath 尝试）
        date_published = selector.xpath('//*[@class="date"]/text()').get()
        if not date_published:
            date_published = selector.xpath('//time/@datetime').get()
        date_published = date_published.strip() if date_published else "N/A"

        # 提取笔记的收藏数量（stars）
        stars = selector.xpath('//*[@class="count"]/text()').get()
        stars = stars.strip() if stars else "0"

        # 提取视频 URL（如果有）
        video_url = selector.xpath('//video/@src').get()
        video_url = video_url.strip() if video_url else "N/A"

        # 提取帖子内容（Caption）
        caption = selector.xpath('//*[@name="description"]/@content').get()
        caption = caption.strip() if caption else "N/A"

        # 更新全局列表中的相应条目
        if note_id in id_list:
            index = id_list.index(note_id)
            commentNr_list[index] = comments
            datePublished_list[index] = date_published
            starNr_list[index] = stars
            video_urls[index] = video_url
            caption_list[index] = caption
            print(f"已提取附加字段，笔记ID: {note_id}")
        else:
            print(f"笔记ID {note_id} 未在 id_list 中找到。")

    except Exception as e:
        print(f"提取附加字段时出错，笔记ID: {note_id}, 错误: {str(e)}")

def download_image(url, save_path):
    """
    下载图片并保存到指定路径。
    
    Args:
        url (str): 图片的 URL 地址
        save_path (str): 图片保存的本地路径
    
    Returns:
        bool: 下载是否成功
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
        return False

def create_directories():
    """
    创建用于存储主图片和头像图片的目录。
    """
    if not os.path.exists('images_user'):
        os.makedirs('images_user')
    if not os.path.exists('avatars_user'):
        os.makedirs('avatars_user')

# 创建目录以存储图片
create_directories()

# 定义进度条用于跟踪已爬取的笔记数量
qbar = tqdm(total=num, desc="爬取进度")

# 开始下拉加载页面直至达到所需笔记数量
while len(id_list) < num:
    for _ in range(3):
        browser.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)

    parseUserPage(browser.page_source)

    # 检测是否到达页面末尾（根据页面提示文本判断）
    if '- THE END -' in browser.page_source or 'No more content' in browser.page_source:
        print(f"已到达内容末尾。总共收集: {len(id_list)} 条")
        break

    time.sleep(random.uniform(2, 4))

print(f"总共收集的条目数: {len(id_list)}")
if id_list:
    print("收集的数据样本:")
    for i in range(min(5, len(id_list))):
        print(f"URL: {id_list[i]}")

# 如果收集的条目超过用户所需数量，则截断各个列表
if len(id_list) > num:
    id_list = id_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    noteURL_list = noteURL_list[:num]
    commentNr_list = commentNr_list[:num]
    post_title_list = post_title_list[:num]
    datePublished_list = datePublished_list[:num]
    images_list = images_list[:num]
    author_avatar_list = author_avatar_list[:num]
    starNr_list = starNr_list[:num]
    authorCollectNr_list = authorCollectNr_list[:num]
    authorFansNr_list = authorFansNr_list[:num]
    authorNoteNr_list = authorNoteNr_list[:num]
    video_urls = video_urls[:num]
    caption_list = caption_list[:num]

print(f"截断后的总条目数: {len(id_list)}")
print("收集的数据样本:")
for i in range(min(5, len(id_list))):
    print(f"作者: {authorName_list[i]}, 点赞: {likeNr_list[i]}, URL: {id_list[i]}")

qbar.close()

# 提取每个笔记的附加字段（例如帖子内容、发布时间和评论数量）
print("开始提取附加字段，包括帖子内容、日期发布和评论数量...")
qbar = tqdm(total=len(id_list), desc="已获取的笔记数量...")

for note_id, note_url in zip(id_list, noteURL_list):
    parseNotePage(note_url, note_id)
    qbar.update(1)
    time.sleep(random.uniform(2, 4))  # 礼貌等待，避免服务器过载

qbar.close()

# 下载主图片和头像图片
print("开始下载主图片和头像图片...")

# 下载主图片
image_download_bar = tqdm(total=len(images_list), desc="下载主图片")
for idx, image_url in enumerate(images_list):
    if image_url == "N/A":
        image_download_bar.update(1)
        continue
    # 构造图片保存路径，使用 note_id 作为文件名
    image_extension = os.path.splitext(image_url)[1].split('?')[0]
    if image_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        image_extension = '.jpg'
    image_filename = f"images_user/{id_list[idx]}{image_extension}"
    success = download_image(image_url, image_filename)
    if not success:
        image_filename = "N/A"
    images_list[idx] = image_filename
    image_download_bar.update(1)
image_download_bar.close()

# 下载头像图片
avatar_download_bar = tqdm(total=len(author_avatar_list), desc="下载头像图片")
for idx, avatar_url in enumerate(author_avatar_list):
    if avatar_url == "N/A":
        avatar_download_bar.update(1)
        continue
    avatar_extension = os.path.splitext(avatar_url)[1].split('?')[0]
    if avatar_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        avatar_extension = '.jpg'
    avatar_filename = f"avatars_user/{id_list[idx]}{avatar_extension}"
    success = download_image(avatar_url, avatar_filename)
    if not success:
        avatar_filename = "N/A"
    author_avatar_list[idx] = avatar_filename
    avatar_download_bar.update(1)
avatar_download_bar.close()

print("所有图片下载完成。")

# 将数据整理为字典（包括“Author Avatar”与“Caption”字段）
data = {
    'Author Name': authorName_list,
    'Likes': likeNr_list,
    'Comments': commentNr_list,
    'Post Title': post_title_list, 
    'Caption': caption_list,
    'Date Published': datePublished_list,
    'Images': images_list,          # 主图片的本地路径
    'Author Avatar': author_avatar_list,  # 头像图片的本地路径
    'Stars': starNr_list,
    'Author Collect Nr': authorCollectNr_list,
    'Author Fans Nr': authorFansNr_list,
    'Author Note Nr': authorNoteNr_list,
    'Video URL': video_urls,
    'Post ID': id_list,
    'Note URL': noteURL_list
}

# 将数据存储为 DataFrame 并导出 CSV
df = pd.DataFrame(data)
# 若你想以某个字段作为索引，请确保该字段在 data 中存在
df.set_index('Post ID', inplace=True)
print(df.head())
df.to_csv('scraped_xhs_user.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_xhs_user.csv'")

爬取进度:   0%|          | 0/100 [00:00<?, ?it/s]

正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 21
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 42
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 42
正在分析页面结构...
找到 21 个内容元素
当前已爬取总数: 42
正在分析页面结构...
找到 21 个内容元素
当前已爬

已获取的笔记数量...:   0%|          | 0/100 [00:00<?, ?it/s]

已提取附加字段，笔记ID: 678a28b50000000017003474
已提取附加字段，笔记ID: 6792206c0000000029034b18
已提取附加字段，笔记ID: 67ab22a5000000002901451f
已提取附加字段，笔记ID: 67a72569000000002903dbee
已提取附加字段，笔记ID: 6792fa6a000000002900d1a9
已提取附加字段，笔记ID: 6792f9d5000000002a00ce51
已提取附加字段，笔记ID: 67936d94000000002902884a
已提取附加字段，笔记ID: 6791e51000000000280350a1
已提取附加字段，笔记ID: 678f5cbf0000000029008b82
已提取附加字段，笔记ID: 678a2ff6000000001901b6ec
已提取附加字段，笔记ID: 67874af50000000019006f5f
已提取附加字段，笔记ID: 6787142b000000001603e621
已提取附加字段，笔记ID: 677f9560000000000b038b88
已提取附加字段，笔记ID: 6777b89a000000000902fcff
已提取附加字段，笔记ID: 67775dee000000001301be86
已提取附加字段，笔记ID: 6773bc94000000001300f133
已提取附加字段，笔记ID: 67740a1700000000140265e9
已提取附加字段，笔记ID: 67736c08000000000902dcfd
已提取附加字段，笔记ID: 676cd4f1000000001301b94a
已提取附加字段，笔记ID: 676be337000000000902e705
已提取附加字段，笔记ID: 676bacb1000000001402505a
已提取附加字段，笔记ID: 6766293c000000000902ec2f
已提取附加字段，笔记ID: 676530c900000000130086e0
已提取附加字段，笔记ID: 6764e385000000001301bda3
已提取附加字段，笔记ID: 6763f35e00000000130181a5
已提取附加字段，笔记ID: 6763e990000

下载主图片:   0%|          | 0/100 [00:00<?, ?it/s]

下载头像图片:   0%|          | 0/100 [00:00<?, ?it/s]

所有图片下载完成。
                         Author Name Likes  Comments           Post Title  \
Post ID                                                                     
678a28b50000000017003474      皇家宠物食品    21  共 14 条评论       新春喵宴！宠物年夜饭🐱皇包了   
6792206c0000000029034b18      皇家宠物食品   241         0   📒体验官招募｜幼猫幼犬，就吃皇家❗️   
67ab22a5000000002901451f      皇家宠物食品    12  共 10 条评论   2.14不止撒糖，还要给主子撒福利！   
67a72569000000002903dbee      皇家宠物食品    36         0  复工精神状态良好...只是想体验猫生🐱   
6792fa6a000000002900d1a9      皇家宠物食品    40  共 69 条评论       大年初一|阿皇来给您拜年啦！   

                                                                    Caption  \
Post ID                                                                       
678a28b50000000017003474  新春来临，阿皇给毛孩子送年夜饭🎁 干粮吃腻了？马上呈上湿粮餐包🍚 小香包和小鲜包香味四溢，饭...   
6792206c0000000029034b18  “新手养宠如何养出健康小猫小狗？” 相信是很多铲屎官面临的难题🤯 毕竟打好身体基础真的重要！...   
67ab22a5000000002901451f  铲屎官紧急集合 🐾 2月14日不止是情人节，更是【皇家宠爱日】👑 下午4点-6点，锁定直播间...   
67a72569000000002903dbee  阿皇复工后的精神状态be like： 身体在工位，灵魂还在